In [15]:
import torch
from fastai.text.all import *
from utils import TextDataLoadersInspector

In [7]:
## Get the data for x
path = untar_data(URLs.IMDB)
files = get_text_files(path, folders=["train", "test"])
files[0]

Path('/home/jp/.fastai/data/imdb/test/neg/12193_3.txt')

In [9]:
## Get data for y and convert y to idx (numericalize)
lbls = files.map(Self.parent.name()).unique()
v2i = lbls.val2idx()
print(v2i)

{'neg': 0, 'pos': 1}


In [24]:
# Build a Dataset for PyTorch (has to support indexing, e.g. __getitem__ and __len__)
class Dataset:
    """Dataset for Pytorch (has to support indexing of data, hence __getitem__ and __len__).
    Input data is a list of file paths"""
    
    def __init__(self, fns):
        self.fns = fns
        
    def __len__(self): 
        return len(self.fns)
    
    def __getitem__(self, i):
        txt = self.fns[i].open().read()
        return txt

    
def random_splitting(valid_pct=0.2, seed=None):
    "Create function that splits items between train/val with valid_pct randomly."
    def _inner(o):
        if seed is not None: torch.manual_seed(seed)
        rand_idx = L(int(i) for i in torch.randperm(len(o)))
        cut = int(valid_pct * len(o))
        return rand_idx[cut:],rand_idx[:cut]
    return _inner

In [31]:
train, valid = random_splitting(valid_pct=0.1)(files) 
train, valid

((#45000) [15250,1570,24325,28918,22094,1260,17504,9946,13916,42368...],
 (#5000) [44264,11387,9413,2895,45988,1730,28326,48469,281,38305...])

In [33]:
valid_ds = Dataset(valid)
valid_ds[0]

AttributeError: 'int' object has no attribute 'open'